<a href="https://colab.research.google.com/github/lqcgithub/MiningofMassiveDatasets/blob/main/TextSimilarityLSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install pyspark

In [ ]:
pip install mmh3

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
import pyspark
from pyspark import SparkConf, SparkContext

In [ ]:
import re
import numpy as np
import collections
import random
from operator import itemgetter

In [ ]:
import nltk
from nltk import ngrams
import mmh3
import time

In [ ]:
conf = SparkConf().setMaster("local").setAppName("Market Basket with Apriori")
sc = SparkContext.getOrCreate(conf=conf)

In [ ]:
baseUrl = "https://tuoitre.vn"
url = "https://congnghe.tuoitre.vn/"

In [ ]:
def crawlTuoitreNews(baseUrl, url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")
  titles = soup.select('h3.title-news')
  links = [link.find('a').attrs["href"] for link in titles]
  data = []
  for link in links:
    news = requests.get(baseUrl + link)
    soup = BeautifulSoup(news.content, "html.parser")
    if (soup.find("h1", class_="article-title")is not None):
      title = soup.find("h1", class_="article-title").text.strip()
      abstract = soup.find("h2", class_="sapo").text.strip()
      body = soup.find("div", id="main-detail-body")
      contents = body.findAll("p", recursive=False)
      contents = [content.text.strip() for content in contents]
      content = ('').join(contents)
      data.append({
          "title": title,
          "abstract": abstract,
          "content": content,
      })
  return data
data = crawlTuoitreNews(baseUrl, url)

In [ ]:
print(data[0]["content"])
print(len(data))

Thế hệ trẻ hiện nay, đặc biệt là GenZ đang sống trong thời đại của các thiết bị di động thông minh. Nhưng TV vẫn đóng vai trò quan trọng của một sản phẩm giải trí không thể thiếu tại nhà. Thống kê cho thấy người dùng vẫn dành một khoảng thời gian đáng kể để thưởng thức các gameshow, phim, YouTube trên TV…Coocaa nổi lên là thương hiệu TV đang chiếm được cảm tình của người dùng, đặc biệt là khách hàng trẻ. Đây trở thành một trong những dòng TV có doanh số đứng đầu sàn thương mại điện tử Lazada tại khu vực Đông Nam Á. Sở dĩ khách hàng yêu thích sản phẩm bởi chất lượng của sản phẩm đem lại vượt trội so với giá thành, tiêu biểu có thể kể đến dòng S6G Pro.Với thông điệp CooLife, Coocaa muốn mang đến nhiều hơn những giá trị giải trí cho khách hàng. Khẩu hiệu này là ghép của Cool và Life, mở ra trải nghiệm mới cho những người trẻ khi TV không còn bị giới hạn bởi những chương trình truyền hình mà còn nhiều hơn thế.Người dùng có thể cùng bạn bè hát karaoke sau một ngày làm việc hăng say, hòa mìn

In [ ]:
def preprocess(document):
    document = re.sub(r'[^\w\s]','',document)
    tokens = document.lower()
    tokens = tokens.split()
    return tokens

In [ ]:
# document = "Trận này, Leicester áp đảo ngay sau tiếng còi khai cuộc. Đây cũng là điều dễ hiểu, bởi đối thủ của họ - Sheffield United là đội bóng đang xếp cuối bảng và gần như cầm chắc vé xuống hạng khi kém nhóm an toàn tới 14 điểm.Phút 12, cơ hội mở tỉ số đến với Leicester sau pha dàn xếp tấn công bên cánh trái. Nhưng may mắn đã không đứng về phía đội chủ nhà khi pha đánh đầu về góc xa của Ayoze Perez lại đưa bóng trúng cột dọc khung thành.  Những phút sau đó, Leicester tiếp tục tấn công và tạo ra thêm nhiều tình huống nguy hiểm nữa. Dù áp đảo hoàn toàn, nhưng phải đến cuối hiệp 1 các học trò HLV Brendan Rodgers mới đưa được bóng vào lưới khung thành thủ môn Aaron Ramsdale.  Phút 39, từ pha lên bóng bên cánh trái, Jamie Vardy bứt tốc xâm nhập vòng cấm đón đường chọc khe của đồng đội rồi thực hiện đường căng ngang vào giữa. Trong tư thế không người kèm, Kelechi Iheanacho dễ dàng ập vào đệm bóng tung lưới thủ môn Aaron Ramsdale từ cự li gần, mở tỉ số cho Leicester.  Bị thủng lưới, Sheffield United đẩy cao đội hình tấn công trong hiệp 2 và cơ hội tiếp tục đến với Leicester khi hàng phòng ngự đội khách để lộ ra nhiều khoảng trống.  Phút 64, sau vài cơ hội bị bỏ lỡ, đội chủ nhà cũng đã có bàn thắng nhân đôi cách biệt nhờ công tiền vệ Ayoze Perez với pha dứt điểm hiểm hóc ngay mép vòng cấm sau tình huống phản công bên cánh phải.  Sau bàn thua này, các cầu thủ Sheffield United càng thi đấu thiếu tập trung, đặc biệt là trong phòng ngự.  Hệ quả, chỉ trong 11 phút từ 69 đến 80, lưới của thủ môn Aaron Ramsdale đã phải rung lên thêm 3 lần nữa sau những cú dứt điểm trong tư thế vô cùng thoải mái của Kelechi Iheanacho (phút 69 và 78) và Jamie Vardy (phút 80), giúp Leicester khép lại trận đấu với chiến thắng cách biệt 5-0.  Thắng trận này, Leicester đã tạm vượt qua Manchester United (chưa đá vòng 28) để vươn lên vị trí thứ hai trên bảng xếp hạng với 56 điểm, hơn Man Utd 2 điểm và kém đội đầu bảng Man City 15 điểm. "
# preprocess(document)[0:5]

In [ ]:
doc_as_shingles = []
for document in data:
  tokens = preprocess(document['content'])
  shingles = ngrams(tokens, 3)
  doc_as_shingles.append(list(shingles))
print(doc_as_shingles[0])

[('thế', 'hệ', 'trẻ'), ('hệ', 'trẻ', 'hiện'), ('trẻ', 'hiện', 'nay'), ('hiện', 'nay', 'đặc'), ('nay', 'đặc', 'biệt'), ('đặc', 'biệt', 'là'), ('biệt', 'là', 'genz'), ('là', 'genz', 'đang'), ('genz', 'đang', 'sống'), ('đang', 'sống', 'trong'), ('sống', 'trong', 'thời'), ('trong', 'thời', 'đại'), ('thời', 'đại', 'của'), ('đại', 'của', 'các'), ('của', 'các', 'thiết'), ('các', 'thiết', 'bị'), ('thiết', 'bị', 'di'), ('bị', 'di', 'động'), ('di', 'động', 'thông'), ('động', 'thông', 'minh'), ('thông', 'minh', 'nhưng'), ('minh', 'nhưng', 'tv'), ('nhưng', 'tv', 'vẫn'), ('tv', 'vẫn', 'đóng'), ('vẫn', 'đóng', 'vai'), ('đóng', 'vai', 'trò'), ('vai', 'trò', 'quan'), ('trò', 'quan', 'trọng'), ('quan', 'trọng', 'của'), ('trọng', 'của', 'một'), ('của', 'một', 'sản'), ('một', 'sản', 'phẩm'), ('sản', 'phẩm', 'giải'), ('phẩm', 'giải', 'trí'), ('giải', 'trí', 'không'), ('trí', 'không', 'thể'), ('không', 'thể', 'thiếu'), ('thể', 'thiếu', 'tại'), ('thiếu', 'tại', 'nhà'), ('tại', 'nhà', 'thống'), ('nhà', 'thốn

In [ ]:
doc_hashed = []

def hash_doc(shingles):
  shingles_hashed = []
  for shingle in shingles:
    hash = mmh3.hash(' '.join(shingle))
    shingles_hashed.append(hash)
  return shingles_hashed  

for document in doc_as_shingles:
  doc_hashed.append(list(hash_doc(document)));
print(doc_hashed[0])

[-722364660, -1343950092, 471844217, 1801623279, 1482432412, -362830372, 1596274497, 1511091060, 1965756164, 1698391027, -1061828549, -2040762716, 989097069, -1357895415, 2023521466, -519954334, 1528694819, 1940699007, -828293076, -2116853031, 1601166066, 684307324, 63188002, 1162830436, -1027326415, -1300567827, -67381483, 1511136605, 1800761602, -185403472, 145247975, -995500379, -64199284, -1818579863, -1255928249, 1218240626, -1044735248, 464577980, 1394130347, 1955738588, -1600619982, -921939991, 2117784948, -1837716976, -28807018, -26876701, 1900278102, -1357008956, 824972299, -1575463586, -672447666, -1549496927, 2078392565, 1676353503, 468584616, -441375209, -397874287, 171482329, -105522771, 1246533378, 680572478, -1099135022, 1095685026, 474709262, 53970356, -1747224955, 1306947234, -317833863, 984816755, -1620819708, -1476003526, 1160757882, -295347504, -1723831185, 1289661019, 1034848267, -993065325, -959384532, -362830372, 1097872074, -91893570, 193594294, -1744600909, -19

In [ ]:
start = time.time()
signatures = []
max_shingle_id = 2 ** 32 - 1

next_prime = 4294967311

def pick_random_coeffs(k):
    rand_list = []

    while k > 0:
        rand_index = random.randint(0, max_shingle_id)

        while rand_index in rand_list:
            rand_index = random.randint(0, max_shingle_id)

        rand_list.append(rand_index)
        k = k - 1
    return rand_list

coeff_a = pick_random_coeffs(10)
coeff_b = pick_random_coeffs(10)

for document in doc_hashed:
  signature =[]
  for i in range(0, 10):
    min_hash_code = next_prime + 1
    for shingle in document: 
      hash_code = (coeff_a[i] * shingle + coeff_b[i]) % next_prime
      if hash_code < min_hash_code:
        min_hash_code = hash_code
    signature.append(min_hash_code)
  signatures.append(signature)
end = time.time()
print(signatures[0])
print(len(signatures),len(signatures[0]))
print("Minhash completed in : {:.4f} sec".format(end - start))

[8978542, 9347148, 5412068, 2475911, 2875374, 2551886, 5619128, 2539775, 18144280, 3775386]
20 10
Minhash completed in : 0.0561 sec


In [ ]:
def jaccardSim(d1,d2):
    return len(d1.intersection(d2))/len(d1.union(d2))   

In [ ]:
# sims = jaccardSim(set(signatures[0]), set(signatures[1])) 
# sims

In [ ]:
def initialize_array_bucket(bands):
    len_buckets = 101
    array_buckets = []
    for band in range(bands):
        array_buckets.append([[] for i in range(len_buckets)])
    return array_buckets

In [ ]:
def apply_LSH_technique(signatures,t,bands,rows):
    if bands * rows != len(signatures):
        print('split to bands error.')

    array_buckets = initialize_array_bucket(bands)

    # hash_funcs = pick_random_coeffs(10);


    candidates = {}
    
    i = 0
    for b in range(bands):
        buckets = array_buckets[b]        
        band = np.array(signatures)[i:i+rows,:]
        for col in range(band.shape[1]):    
            
            key = int(sum(band[:,col]) % len(buckets))
            
            buckets[key].append(col)
        i = i+rows

        for item in buckets:
            if len(item) > 1:
                pair = (item[0], item[1])
                if pair not in candidates:
                    A = np.array(signatures)[:,item[0]]
                    B = np.array(signatures)[:,item[1]]
                    similarity = cosine_distance(A,B)
                    if similarity >= t:
                        candidates[pair] = similarity


    sort = sorted(candidates.items(),key=itemgetter(1), reverse=True)

    return candidates,sort

def euclidean_distance(x,y,r=2.0):
    try:
        
        return sum(((x[i] - y[i]) ** r) for i in range(len(x))) ** (1.0/r)
    
    except (ValueError,ZeroDivisionError):
        print ('Please, enter only even values for "r > 0".')
    except IndexError:
        print ('Please, the sets must have the same size.')

def cosine_distance(x,y):
    prodAB = sum([x[i]*y[i] for i in range(len(x))])
    zeros = [0 for i in range(len(x))]
    A = euclidean_distance(x,zeros)
    B = euclidean_distance(y,zeros)
    return prodAB / (A*B)

start = time.time()
candidates, sort = apply_LSH_technique(signatures, 0.5, 4, 5)
end = time.time()
print(candidates)
print("LSH completed in: {:.4f}".format(end - start))

{(5, 8): 0.6742290167003142, (0, 3): 0.53815939149342, (1, 7): 0.6215590277399905}
LSH completed in: 0.0022


In [ ]:
print(data[5]['abstract'])
print(data[8]['abstract'])

TTO - Được ví như 'bộ não số' trong chính quyền điện tử, trung tâm điều hành thông minh (IOC) đang được nhiều tỉnh thành trong cả nước tập trung triển khai xây dựng.
TTO - CEO Twitter, ông Jack Dorsey, vừa bán đấu giá được phiên bản số dòng tweet đầu tiên trên nền tảng Twitter với giá 2,9 triệu USD (gần 67 tỉ đồng).


In [ ]:
# def crawKenh14NewsData(baseUrl, url):
#     response = requests.get(url)
#     soup = BeautifulSoup(response.content, "html.parser")

#     titles = soup.select('h3.knswli-title')
    
#     links = [link.find('a').attrs["href"] for link in titles]
    
#     data = []
#     for link in links:
#         news = requests.get(link)
#         soup = BeautifulSoup(news.content, "html.parser")
#         title = soup.find("h1", class_="kbwc-title").text
#         abstract = soup.find("h2", class_="knc-sapo").text
#         body = soup.find("div", class_="knc-content")
#         content = ""
#         try:
#             contents = body.select("p", recursive=False)
#             contents = [content.text for content in contents]
#             content = ('').join(contents)
#         except:
#             content = ""
#         data.append({
#             "title": title,
#             "abstract": abstract,
#             "content": content,
#         })
#     return data

# baseUrl = "https://kenh14.vn/"
# url = "http://search.kenh14.vn/?query=covid"

# data = crawKenh14NewsData(baseUrl, url)

In [ ]:
# print(len(data))